In [1]:
(defn set_cell [matrix i j el] (update matrix (+ (* 10 i) j) (fn [_] el)))
(defn get_cell [matrix i j] (get matrix (+ (* 10 i) j)))

#'user/get_cell

In [2]:
(def rules 
    {"" "move_random"
     "up" "move_random"
     "down" "move_random"
     "left" "move_random"
     "right" "move_random"
     "dirty" "suck"
     "dirtyup" "suck"
     "dirtydown" "suck"
     "dirtyleft" "suck"
     "dirtyright" "suck"
     })

#'user/rules

In [3]:
(def fields 
    [
        [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
         -1  0  0  0  0  0  0  0  0 -1 
         -1  0  0  0  0  0  0  0  0 -1 
         -1  0  0  0  0  0  0  0  0 -1 
         -1  0  0  0  0  0  0  0  0 -1
         -1  0  0  0  0  0  0  0  0 -1 
         -1  0  0  0  0  0  0  0  0 -1 
         -1  0  0  0  0  0  0  0  0 -1 
         -1  0  0  0  0  0  0  0  0 -1
         -1 -1 -1 -1 -1 -1 -1 -1 -1 -1] 
        
        [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
         -1  0  0  0  0  0  0  0  0 -1 
         -1  0 -1 -1 -1 -1 -1 -1  0 -1 
         -1  0 -1  0  0  0  0  0  0 -1 
         -1  0 -1  0  0  0  0 -1  0 -1
         -1  0 -1  0  0  0  0 -1  0 -1 
         -1  0 -1  0  0  0  0 -1  0 -1 
         -1  0 -1 -1  0 -1 -1 -1  0 -1 
         -1  0  0  0  0  0  0  0  0 -1
         -1 -1 -1 -1 -1 -1 -1 -1 -1 -1] 
        
        [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1
         -1  0 -1  0  0  0 -1  0  0 -1
         -1  0 -1  0 -1  0 -1  0  0 -1
         -1  0 -1  0 -1  0 -1  0  0 -1
         -1  0 -1  0 -1  0 -1  0  0 -1
         -1  0 -1  0 -1  0 -1  0  0 -1
         -1  0 -1  0 -1  0 -1  0  0 -1
         -1  0 -1  0 -1  0 -1  0  0 -1
         -1  0  0  0 -1  0  0  0  0 -1
         -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
        ])

(defn get_random_env [field_index dirt_prob] 
    {:field (get fields field_index)
     :activate_bump ""
     :agent_x 5
     :agent_y 5
     :dirt_prob dirt_prob
     })
(defn is_dirty [env x y] (< 0 (get_cell (:field env) x y)))
(defn is_wall [env x y] (= -1 (get_cell (:field env) x y)))
(defn get_dirt_level [field] (/ (reduce + (vec (map (fn [e] (if (= e -1) 0 e)) field))) 100))

(defn apply_action [env action] 
    (cond 
        (not (= (:activate_bump env) ""))
            (apply_action (update env :activate_bump (fn [_] "")) action)
        (= action "suck")
            (cond
                (is_dirty env (:agent_x env) (:agent_y env))
                (update env 
                    :field (fn [_] 
                       (set_cell
                           (:field env) 
                           (:agent_x env) 
                           (:agent_y env) 
                               (- (get_cell
                                    (:field env) 
                                    (:agent_x env) 
                                    (:agent_y env)) 1))))
                :else env
                )
        (= action "move_up") 
            (if (is_wall env (- (:agent_x env) 1) (:agent_y env))
                (update env :activate_bump (fn [_] "up"))
                (update env :agent_x - 1)
                )
        (= action "move_down") 
            (if (is_wall env (+ (:agent_x env) 1) (:agent_y env))
                (update env :activate_bump (fn [_] "down"))
                (update env :agent_x + 1)
                )
        (= action "move_left") 
            (if (is_wall env (:agent_x env) (- (:agent_y env) 1))
                (update env :activate_bump (fn [_] "left"))
                (update env :agent_y - 1)
                )
        (= action "move_right") 
            (if (is_wall env (:agent_x env) (+ (:agent_y env) 1))
                (update env :activate_bump (fn [_] "right"))
                (update env :agent_y + 1)
                )
        (= action "idle") env
        ))

(defn step [env, ag]
    (apply_action
        (update env :field (fn [_] (vec (apply list (map #(if (= %1 -1) -1 (+ %1 %2)) (:field env) (vec (apply list (repeatedly 100 (fn [] (if (< (rand) (:dirt_prob env)) 1 0))))))))))
         ((:activate_sensors ag) ag (is_dirty env (:agent_x env) (:agent_y env)) (:activate_bump env))))

#'user/step

In [4]:
(defn idle [ag & args] "idle")
(defn suck [ag & args] "suck")
(defn move_up [ag & args] "move_up")
(defn move_down [ag & args] "move_down")
(defn move_left [ag & args] "move_left")
(defn move_right [ag & args] "move_right")
(defn move_random [ag & args] 
    ((fn f[forb, arr] 
    ((fn [ch] (if 
                  (some true? (vec (map (fn [e] (= e (get arr ch))) forb))) 
                  (f forb arr) 
                  (get arr ch)
                  )) 
                 (rand-int (count arr)) )
    ) (vec (map (fn [e] (str "move_" e)) args)) ["move_left" "move_up" "move_right" "move_down"]) )

(defn get_reflex_agent [rules, init_energy]
    {
        :rules (apply hash-map (flatten (map (fn [entry] [(first entry) (resolve (symbol (last entry)))]) rules)))
        :energy init_energy
        :activate_sensors (fn [ag dirty bump] 
            (cond
                (< (:energy ag) 2) "idle"
                true ((get (:rules ag) (apply str [(if dirty "dirty" "") bump])) ag)
                ))
        })

(defn get_reflex_model_agent [rules, init_energy]
    {
        :rules (apply hash-map (flatten (map (fn [entry] [(first entry) (resolve (symbol (last entry)))]) rules)))
        :energy init_energy
        :last_action ""
        :activate_sensors (fn [ag dirty bump] 
            (cond
                (< (:energy ag) 2) "idle"
                true ((get (:rules ag) (apply str [(if dirty "dirty" "") bump])) ag (clojure.string/replace (:last_action ag) #"move_" ""))
                ))
        })

#'user/get_reflex_model_agent

In [5]:
(prn "Simple reflex agent and field 0")
(def e (get_random_env 0 0.005))
(def a (get_reflex_agent rules 2000))
(prn (vec (map (fn [_] (first [(get_dirt_level (:field e)) (def e (step e a))]) ) (range 100))))

(prn "Reflex model agent and field 0")
(def e (get_random_env 0 0.005))
(def a (get_reflex_model_agent rules 2000))
(prn (vec (map (fn [_] (first [(get_dirt_level (:field e)) (def e (step e a))]) ) (range 100))))

(prn "Simple reflex agent and field 1")
(def e (get_random_env 1 0.005))
(def a (get_reflex_agent rules 2000))
(prn (vec (map (fn [_] (first [(get_dirt_level (:field e)) (def e (step e a))]) ) (range 100))))

(prn "Reflex model agent and field 1")
(def e (get_random_env 1 0.005))
(def a (get_reflex_model_agent rules 2000))
(prn (vec (map (fn [_] (first [(get_dirt_level (:field e)) (def e (step e a))]) ) (range 100))))

(prn "Simple reflex agent and field 2")
(def e (get_random_env 2 0.005))
(def a (get_reflex_agent rules 2000))
(prn (vec (map (fn [_] (first [(get_dirt_level (:field e)) (def e (step e a))]) ) (range 100))))

(prn "Reflex model agent and field 2")
(def e (get_random_env 2 0.005))
(def a (get_reflex_model_agent rules 2000))
(prn (vec (map (fn [_] (first [(get_dirt_level (:field e)) (def e (step e a))]) ) (range 100))))

"Simple reflex agent and field 0"
[0 0 0 0 0 1/100 1/100 1/100 1/50 1/50 1/50 1/50 1/20 3/50 7/100 7/100 7/100 7/100 7/100 2/25 9/100 9/100 9/100 9/100 9/100 9/100 9/100 9/100 9/100 1/10 1/10 13/100 7/50 7/50 7/50 7/50 3/20 3/20 3/20 3/20 3/20 3/20 3/20 3/20 7/50 3/20 17/100 17/100 17/100 17/100 17/100 9/50 1/5 21/100 21/100 21/100 21/100 21/100 21/100 21/100 21/100 21/100 11/50 23/100 23/100 23/100 23/100 23/100 23/100 23/100 23/100 23/100 23/100 1/4 1/4 1/4 1/4 1/4 13/50 13/50 13/50 13/50 13/50 13/50 13/50 27/100 7/25 7/25 7/25 29/100 29/100 29/100 3/10 29/100 29/100 3/10 8/25 8/25 8/25 8/25]
"Reflex model agent and field 0"
[0 1/100 0 0 1/100 1/100 1/100 1/100 1/50 1/50 1/50 3/100 3/100 3/100 1/25 1/25 1/25 1/20 3/50 3/50 7/100 9/100 9/100 1/10 1/10 1/10 1/10 1/10 1/10 11/100 11/100 11/100 13/100 13/100 13/100 7/50 7/50 7/50 7/50 7/50 7/50 7/50 3/20 3/20 3/20 3/20 3/20 4/25 4/25 4/25 17/100 17/100 17/100 4/25 4/25 4/25 17/100 9/50 17/100 17/100 17/100 17/100 17/100 17/100 17/100 17/